In [2]:
import os
import yaml
import cosmo
import numpy
import itertools
from glob import glob
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from cosmo.filesystem import find_files, data_from_exposures, data_from_jitters
from cosmo.monitor_helpers import absolute_time, explode_df
from monitorframe.datamodel import BaseDataModel
from monitorframe.monitor import BaseMonitor

In [3]:
with open(os.environ['COSMO_CONFIG']) as yamlfile:
    SETTINGS = yaml.safe_load(yamlfile)

FILES_SOURCE = SETTINGS['filesystem']['source']
COS_MONITORING = SETTINGS['output']

In [4]:
testfile = FILES_SOURCE + "/15776/le090edhq_corrtag.fits.gz"
with fits.open(testfile) as f:
    f.info()
    data = f["EVENTS"].data
    timeline = f["TIMELINE"].data
    
data.columns  
timeline.columns

Filename: /grp/hst/cos2/cosmo/15776/le090edhq_corrtag.fits.gz
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     166   ()      
  1  EVENTS        1 BinTableHDU    250   1440318R x 12C   [1E, 1I, 1I, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1I, 1B]   
  2  GTI           1 BinTableHDU     22   1R x 2C   [1D, 1D]   
  3  TIMELINE      1 BinTableHDU    264   1331R x 12C   [1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E, 1E]   


ColDefs(
    name = 'TIME'; format = '1E'; unit = 's'; disp = 'F8.3'
    name = 'LONGITUDE'; format = '1E'; unit = 'degree'; disp = 'F10.6'; coord_type = 'RA---TAN'; coord_unit = 'deg'; coord_ref_point = 512.0; coord_ref_value = 189.955952071
    name = 'LATITUDE'; format = '1E'; unit = 'degree'; disp = 'F10.6'; coord_type = 'DEC--TAN'; coord_unit = 'deg'; coord_ref_point = 512.0; coord_ref_value = -52.86080947004
    name = 'SUN_ALT'; format = '1E'; unit = 'degree'; disp = 'F6.2'
    name = 'SUN_ZD'; format = '1E'; unit = 'degree'; disp = 'F6.2'
    name = 'TARGET_ALT'; format = '1E'; unit = 'degree'; disp = 'F6.2'
    name = 'RADIAL_VEL'; format = '1E'; unit = 'km /s'; disp = 'F7.5'
    name = 'SHIFT1'; format = '1E'; unit = 'pixel'; disp = 'F7.3'
    name = 'LY_ALPHA'; format = '1E'; unit = 'count /s'; disp = 'G15.6'
    name = 'OI_1304'; format = '1E'; unit = 'count /s'; disp = 'G15.6'
    name = 'OI_1356'; format = '1E'; unit = 'count /s'; disp = 'G15.6'
    name = 'DARKRATE'; for

In [6]:
class NUVDarkDataModel(BaseDataModel):
    """Datamodel for NUV Dark files."""
    files_source = FILES_SOURCE
    subdir_pattern = '?????'

    def get_new_data(self):
        header_request = {
            0: ['ROOTNAME'],
            1: ['EXPSTART','EXPTIME']
            }

        table_request = {
            1: ['TIME','XCORR','YCORR'],
            3: ['TIME','LATITUDE','LONGITUDE','DARKRATE']
            }

        # any special data requests
        # TODO: add spt support for temp, sun_lat, sun_long
        # TODO: is this a good place to add solar data scraping in the future?

        # this is temporary to find the files from the dark programs until
        # we can add the dark files to the monitor_data database
        files = []
        # 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17
        program_ids = ['15776/', '15538/', '14942/', '14521/', '14442/', '13974/', 
                       '13528/', '13126/', '12720/', '12420/', '11894/']
#         program_ids = ['15538/']
        for program in program_ids:
            new_files_source = os.path.join(FILES_SOURCE, program)
            subfiles = glob(os.path.join(new_files_source, "*corrtag*"))
            files += subfiles

        if not files:  # No new files
            return pd.DataFrame()

        # need to add any other keywords that need to be set
        data_results = data_from_exposures(files,
                                           header_request=header_request,
                                           table_request=table_request)

        return data_results

In [ ]:
nuv_model = NUVDarkDataModel()
data_results = nuv_model.get_new_data()
data_results

In [ ]:
class NUVDarkMonitor(BaseMonitor):
    data_model = NUVDarkDataModel
    # TODO: update docs
    # docs = "https://spacetelescope.github.io/cosmo/monitors.html#<darks>"
    output = COS_MONITORING

    run = 'monthly'
        
    def get_data(self):
        # access data, perform any filtering required for analysis
        data = self.model.new_data
        dark_rate_column = []
        dec_year_column = []
        
        xlim = [0, 1024]
        ylim = [0, 1024]
                
        # parallelize, this is going to get bad when looking at a lot of data
        for index, row in data.iterrows():
            subdf = pd.DataFrame({
                "EXPSTART": row["EXPSTART"],
                "TIME": [row["TIME"]],
                "XCORR": [row["XCORR"]],
                "YCORR": [row["YCORR"]],
                "TIME_3": [row["TIME_3"]]
            })
            
            # this is temporary until i understand james' suggestion to use df.apply and lambda functions
            xcorr = subdf["XCORR"][0]
            ycorr = subdf["YCORR"][0]
            filtered_xcorr = xcorr[np.where((xcorr > xlim[0]) & (xcorr < xlim[1]))]
            filtered_ycorr = ycorr[np.where((ycorr > ylim[0]) & (ycorr < ylim[1]))]
            subdf["XCORR"] = [filtered_xcorr]
            subdf["YCORR"] = [filtered_ycorr]
            
            dark_rate_array, dec_year_array = self.calculate_dark_rate(subdf, xlim, ylim)
            dark_rate_column.append(dark_rate_array)
            dec_year_column.append(dec_year_array)
         
        data["DARK_RATE"] = dark_rate_column
        data["DECIMAL_YEAR"] = dec_year_column
        
        # when the monitor method of the monitor is called, it will initialize the self.data attribute
        # with this method and then can be used by the other methods
        return data
    
    def calculate_dark_rate(self, dataframe, xlim, ylim):
        # calculate dark rate for one exposure, with a dataframe with TIME, XCORR, and YCORR values
        
        # need to set this somewhere
        timestep = 25
        time_bins = dataframe["TIME_3"][0][::timestep]
        
        counts = np.histogram(dataframe["TIME"][0], bins=time_bins)[0]
        npix = float((xlim[1] - xlim[0]) * (ylim[1] - ylim[0]))
        dark_rate_array = counts / npix / timestep
        # save the whole histogram in time bins, and then plot each of them
        
        # make a decimal year array corresponding to the time bins of the dark rates
        # do this with the expstart (mjd) and time array from the timeline extension
        # taking the expstart, binning the time array by the timestep, 
        # removing the last element in the array (bin by front edge),
        # and then multiplying by the conversion factor
        # this is done by the absolute_time helper function
        mjd_array = absolute_time(expstart=dataframe['EXPSTART'][0], time=dataframe['TIME_3'][0][::timestep][:-1])
        dec_year_array = mjd_array.decimalyear

        return dark_rate_array, dec_year_array

    def track(self):
        # track something. perhaps current dark rate?
        if self.data is None:
            self.data = self.get_data()
            
        plotdf = self.data[['DECIMAL_YEAR', 'DARK_RATE']]
        
        # i can't get explode_df to work so this is for now
        # i think i know why it doesn't work and i fixed it i just haven't switched to using it
        all_dec_year = []
        all_dark_rates = []
        for index, row in plotdf.iterrows():
            all_dec_year = list(itertools.chain(all_dec_year, row["DECIMAL_YEAR"]))
            all_dark_rates = list(itertools.chain(all_dark_rates, row["DARK_RATE"]))
            
        dark_counts = np.asarray(all_dark_rates)
        fig = plt.figure(figsize=(12, 9))
        # bin_size = 1e-5
        # n_bins = int((dark_counts.max() - dark_counts.min()) / bin_size)
        n_bins = 30
        ax = fig.add_subplot(2, 1, 1)
        ax.hist(dark_counts, bins=n_bins, align='mid', histtype='stepfilled')
        counts, bins = np.histogram(dark_counts, bins=n_bins)
        cuml_dist = np.cumsum(counts)
        count_99 = abs(cuml_dist / float(cuml_dist.max()) - .99).argmin()
        count_95 = abs(cuml_dist / float(cuml_dist.max()) - .95).argmin()

        mean = dark_counts.mean()
        med = np.median(dark_counts)
        std = dark_counts.std()
        mean_obj = ax.axvline(x=mean, lw=2, ls='--', color='r', label='Mean ')
        med_obj = ax.axvline(x=med, lw=2, ls='-', color='r', label='Median')
        two_sig = ax.axvline(x=med + (2 * std), lw=2, ls='-', color='gold')
        three_sig = ax.axvline(x=med + (3 * std), lw=2, ls='-', color='DarkOrange')
        dist_95 = ax.axvline(x=bins[count_95], lw=2, ls='-', color='LightGreen')
        dist_99 = ax.axvline(x=bins[count_99], lw=2, ls='-', color='DarkGreen')

        ax.grid(True, which='both')
        ax.set_title('Histogram of Dark Rates', fontsize=15, fontweight='bold')
        ax.set_ylabel('Frequency', fontsize=15, fontweight='bold')
        ax.set_xlabel('Counts/pix/sec', fontsize=15, fontweight='bold')
        ax.set_xlim(dark_counts.min(), dark_counts.max())
        ax.xaxis.set_major_formatter(FormatStrFormatter('%3.2e'))

        ax = fig.add_subplot(2, 1, 2)
        # log_bins = np.logspace(np.log10(dark.min()), np.log10(dark.max()), 100)
        ax.hist(dark_counts, bins=n_bins, align='mid', log=True, histtype='stepfilled')

        ax.axvline(x=mean, lw=2, ls='--', color='r', label='Mean')
        ax.axvline(x=med, lw=2, ls='-', color='r', label='Median')
        ax.axvline(x=med + (2 * std), lw=2, ls='-', color='gold')
        ax.axvline(x=med + (3 * std), lw=2, ls='-', color='DarkOrange')
        ax.axvline(x=bins[count_95], lw=2, ls='-', color='LightGreen')
        ax.axvline(x=bins[count_99], lw=2, ls='-', color='DarkGreen')

        # ax.set_xscale('log')
        ax.grid(True, which='both')
        ax.set_ylabel('Log Frequency', fontsize=15, fontweight='bold')
        ax.set_xlabel('Counts/pix/sec', fontsize=15, fontweight='bold')
        ax.set_xlim(dark_counts.min(), dark_counts.max())
        ax.xaxis.set_major_formatter(FormatStrFormatter('%3.2e'))

        fig.legend([med_obj, mean_obj, two_sig, three_sig, dist_95, dist_99],
                   ['Median: {0:.2e}'.format(med),
                    'Mean: {0:.2e}'.format(mean),
                    r'2$\sigma$: {0:.2e}'.format(med + (2 * std)),
                    r'3$\sigma$: {0:.2e}'.format(med + (3 * std)),
                    r'95$\%$: {0:.2e}'.format(bins[count_95]),
                    r'99$\%$: {0:.2e}'.format(bins[count_99])],
                   shadow=True,
                   numpoints=1,
                   bbox_to_anchor=[0.8, 0.8])


    def plot(self):
        # select the important columns from the dataframe
        if self.data is None:
            self.data = self.get_data()
            
        plotdf = pd.DataFrame({
            "DECIMAL_YEAR": self.data["DECIMAL_YEAR"],
            "DARK_RATE": self.data["DARK_RATE"]
        })
        
        # i can't get explode_df to work so this is for now
        all_dec_year = []
        all_dark_rates = []
        for index, row in plotdf.iterrows():
            all_dec_year = list(itertools.chain(all_dec_year, row["DECIMAL_YEAR"]))
            all_dark_rates = list(itertools.chain(all_dark_rates, row["DARK_RATE"]))
        
        # not sure what is happening here tbh, still figuring it out
#         self.x = all_dec_year
#         self.y = all_dark_rates
        
#         self.basic_scatter()
        
        fig = plt.figure(figsize=(12, 9))
        plt.scatter(all_dec_year, all_dark_rates)
        # plt.xlim(min(all_dec_year), max(all_dec_year))
        plt.ylim(0, max(all_dark_rates) + 0.5e-5)
        plt.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))
        plt.ticklabel_format(axis='x', style='plain')
        plt.xlabel("Decimal Year")
        plt.ylabel("Dark Rate (c/p/s)")
        plt.grid(True)

    def store_results(self):
        # need to store results if not going in the database
        pass

nuv_monitor = NUVDarkMonitor()

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14521/ldd50xbpq_corrtag.fits.gz
[Errno 6] Device not configured

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13528/lce40tvrq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13528/lce40tvrq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13126/lc4d0rxyq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13126/lc4d0rxyq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r3daiq_corrtag.fits.gz
[Errno 6] Device not configured

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14942/ldq11ch1q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14942/ldq11ch1q_corrtag.fits.gz'


/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13126/lc4d1dqtq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13126/lc4d1dqtq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14442/ld0w13a4q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14442/ld0w13a4q_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/15538/ldvm0wezq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/15538/ldvm0wezq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r1af7q_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r1af7q_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/12720

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/15538/ldvm1fb4q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/15538/ldvm1fb4q_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/12720/lbuhg2nhq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/12720/lbuhg2nhq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r1ge3q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r1ge3q_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14521/ldd518cwq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14521/ldd518cwq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/12420/lbmuk1anq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/12420/lbmuk1anq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14442/ld0w1ccwq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14442/ld0w1ccwq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/15776/le0906ewq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/15776/le0906ewq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13126/lc4d0xtoq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13126/lc4d0xtoq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/15776/le090

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/15538/ldvm12d2q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/15538/ldvm12d2q_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r2jcwq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r2jcwq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r1yqzq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r1yqzq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/12720/lbuhv1mvq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/12720/lbuhv1mvq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/12420/lbmuf

...

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r0rbtq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r0rbtq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14442/ld0w0oh2q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14442/ld0w0oh2q_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13974/lcqf0jxsq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13974/lcqf0jxsq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14942/ldq113fvq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14942/ldq113fvq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14442

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14442/ld0w0ldeq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14442/ld0w0ldeq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13974/lcqf03vjq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13974/lcqf03vjq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13126/lc4d1gbcq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13126/lc4d1gbcq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/15538/ldvm0zasq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/15538/ldvm0zasq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13528/lce40cg0q_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13528/lce40cg0q_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13528/lce40ex4q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13528/lce40ex4q_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13528/lce40mykq_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13528/lce40mykq_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r17cbq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r17cbq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/12720/lbuhp

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r29a2q_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r29a2q_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r3fawq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r3fawq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/12420/lbmus2f2q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/12420/lbmus2f2q_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13126/lc4d06tvq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/13126/lc4d06tvq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14942/ldq105duq_c

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r0za9q_corrtag.fits.gz
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r0za9q_corrtag.fits.gz'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14442/ld0w0jjcq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14442/ld0w0jjcq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/14942/ldq10fe5q_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/14942/ldq10fe5q_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/11894/lb8r30gaq_corrtag.fits
[Errno 2] No such file or directory: '/grp/hst/cos2/cosmo/11894/lb8r30gaq_corrtag.fits'

/user/cmagness/monitors/cosmo/cosmo/filesystem.py:318: Warning:

Bad file found: /grp/hst/cos2/cosmo/13126/lc4d1gbcq_c

In [ ]:
# nuv_monitor.model.new_data
# nuv_monitor.get_data()
nuv_monitor.plot()
nuv_monitor.track()
# nuv_monitor.monitor()

In [ ]:
class NUVDarkMonitor(BaseMonitor):
    data_model = NUVDarkDataModel
    # TODO: update docs
    # docs = "https://spacetelescope.github.io/cosmo/monitors.html#<darks>"
    output = COS_MONITORING

    run = 'monthly'
        
    def get_data(self):
        # access data, perform any filtering required for analysis
        data = self.model.new_data
        
        xlim = (0, 1024)
        ylim = (0, 1024)
                
        data = self.get_dark_rate(data)
        
        # when the monitor method of the monitor is called, it will initialize the self.data attribute
        # with this method and then can be used by the other methods
        return data
    
    def get_dark_rate(df, x_range=(0, 1024), y_range=(0, 1024), time_step=25):
        """Calculate the dark rate per input DataFrame row which should correspond to the 
        data from a single dark."""
        
        def _calculate_darkrate_per_row(row):
            time_bins = row['TIME_3'][::time_step]
            npix = (x_range[1] - x_range[0]) * (y_range[1] - y_range[0])
            
            filtered_time = row.TIME[
                (row.XCORR > x_range[0])
                & (row.XCORR < x_range[1])
                & (row.YCORR > y_range[0])
                & (row.YCORR < y_range[1])
            ]
            counts, _ = np.histogram(filtered_time, bins=time_bins)
            
            return counts / npix / time_step
        
        def _calculate_datetime_bins_per_row(row):
            time_bins = row['TIME_3'][::time_step]
            
            return absolute_time(expstart=row['EXPSTART'], time=time_bins).to_datetime()[:-1]
        
        df['DARK_RATE'] = df.apply(_calculate_darkrate_per_row, axis=1)
        df['DR_TIME'] = df.apply(_calculate_datetime_bins_per_row, axis=1)
        
        return df

    def track(self):
        # track something. perhaps current dark rate?
        if self.data is None:
            self.data = self.get_data()
            
        plotdf = self.data[['DECIMAL_YEAR', 'DARK_RATE']]
        
        # i can't get explode_df to work so this is for now
        # i think i know why it doesn't work and i fixed it i just haven't switched to using it
        all_dec_year = []
        all_dark_rates = []
        for index, row in plotdf.iterrows():
            all_dec_year = list(itertools.chain(all_dec_year, row["DECIMAL_YEAR"]))
            all_dark_rates = list(itertools.chain(all_dark_rates, row["DARK_RATE"]))
            
        dark_counts = np.asarray(all_dark_rates)
        fig = plt.figure(figsize=(12, 9))
#         bin_size = 1e-8
#         n_bins = int((dark_counts.max() - dark_counts.min()) / bin_size)
        n_bins = 30
        ax = fig.add_subplot(2, 1, 1)
        ax.hist(dark_counts, bins=n_bins, align='mid', histtype='stepfilled')
        counts, bins = np.histogram(dark_counts, bins=n_bins)
        cuml_dist = np.cumsum(counts)
        count_99 = abs(cuml_dist / float(cuml_dist.max()) - .99).argmin()
        count_95 = abs(cuml_dist / float(cuml_dist.max()) - .95).argmin()

        mean = dark_counts.mean()
        med = np.median(dark_counts)
        std = dark_counts.std()
        mean_obj = ax.axvline(x=mean, lw=2, ls='--', color='r', label='Mean ')
        med_obj = ax.axvline(x=med, lw=2, ls='-', color='r', label='Median')
        two_sig = ax.axvline(x=med + (2 * std), lw=2, ls='-', color='gold')
        three_sig = ax.axvline(x=med + (3 * std), lw=2, ls='-', color='DarkOrange')
        dist_95 = ax.axvline(x=bins[count_95], lw=2, ls='-', color='LightGreen')
        dist_99 = ax.axvline(x=bins[count_99], lw=2, ls='-', color='DarkGreen')

        ax.grid(True, which='both')
        ax.set_title('Histogram of Dark Rates', fontsize=15, fontweight='bold')
        ax.set_ylabel('Frequency', fontsize=15, fontweight='bold')
        ax.set_xlabel('Counts/pix/sec', fontsize=15, fontweight='bold')
        ax.set_xlim(dark_counts.min(), dark_counts.max())
        ax.xaxis.set_major_formatter(FormatStrFormatter('%3.2e'))

        ax = fig.add_subplot(2, 1, 2)
        # log_bins = np.logspace(np.log10(dark.min()), np.log10(dark.max()), 100)
        ax.hist(dark_counts, bins=n_bins, align='mid', log=True, histtype='stepfilled')

        ax.axvline(x=mean, lw=2, ls='--', color='r', label='Mean')
        ax.axvline(x=med, lw=2, ls='-', color='r', label='Median')
        ax.axvline(x=med + (2 * std), lw=2, ls='-', color='gold')
        ax.axvline(x=med + (3 * std), lw=2, ls='-', color='DarkOrange')
        ax.axvline(x=bins[count_95], lw=2, ls='-', color='LightGreen')
        ax.axvline(x=bins[count_99], lw=2, ls='-', color='DarkGreen')

        # ax.set_xscale('log')
        ax.grid(True, which='both')
        ax.set_ylabel('Log Frequency', fontsize=15, fontweight='bold')
        ax.set_xlabel('Counts/pix/sec', fontsize=15, fontweight='bold')
        ax.set_xlim(dark_counts.min(), dark_counts.max())
        ax.xaxis.set_major_formatter(FormatStrFormatter('%3.2e'))

        fig.legend([med_obj, mean_obj, two_sig, three_sig, dist_95, dist_99],
                   ['Median: {0:.2e}'.format(med),
                    'Mean: {0:.2e}'.format(mean),
                    r'2$\sigma$: {0:.2e}'.format(med + (2 * std)),
                    r'3$\sigma$: {0:.2e}'.format(med + (3 * std)),
                    r'95$\%$: {0:.2e}'.format(bins[count_95]),
                    r'99$\%$: {0:.2e}'.format(bins[count_99])],
                   shadow=True,
                   numpoints=1,
                   bbox_to_anchor=[0.8, 0.8])


    def plot(self):
        # select the important columns from the dataframe
        if self.data is None:
            self.data = self.get_data()
            
        plotdf = pd.DataFrame({
            "DECIMAL_YEAR": self.data["DECIMAL_YEAR"],
            "DARK_RATE": self.data["DARK_RATE"]
        })
        
        # i can't get explode_df to work so this is for now
        all_dec_year = []
        all_dark_rates = []
        for index, row in plotdf.iterrows():
            all_dec_year = list(itertools.chain(all_dec_year, row["DECIMAL_YEAR"]))
            all_dark_rates = list(itertools.chain(all_dark_rates, row["DARK_RATE"]))
        
        # not sure what is happening here tbh, still figuring it out
#         self.x = all_dec_year
#         self.y = all_dark_rates
        
#         self.basic_scatter()
        
        fig = plt.figure(figsize=(12, 9))
        plt.scatter(all_dec_year, all_dark_rates)
        # plt.xlim(min(all_dec_year), max(all_dec_year))
        plt.ylim(0, max(all_dark_rates))
        plt.ticklabel_format(axis='y', style='sci', scilimits=(-2, 2))
        plt.ticklabel_format(axis='x', style='plain')
        plt.xlabel("Decimal Year")
        plt.ylabel("Dark Rate (c/p/s)")
        plt.grid(True)

    def store_results(self):
        # need to store results if not going in the database
        pass

nuv_monitor = NUVDarkMonitor()

In [ ]:
nuv_monitor.plot()
nuv_monitor.track()